# Test Service

Intended to test the service.py evaluator.
Runs the service.py and a simple client.



# Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import sys
import os
import redis
import subprocess as sp
import shlex
import time
import importlib_resources as ir
import socket
from contextlib import closing
import uuid
import pickle

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from flatland.core.env_observation_builder import DummyObservationBuilder
from flatland.envs.persistence import RailEnvPersister
from flatland.evaluators.client import FlatlandRemoteClient
from flatland.evaluators.client import TimeoutException
from flatland.envs.rail_env import RailEnvActions

In [ ]:
  def check_socket(host, port):
    with closing(socket.socket(socket.AF_INET, socket.SOCK_STREAM)) as sock:
        assert sock.connect_ex((host, port)) == 0, f"Port {host} {port} is not open"
        
        print(f"Port {host} {port} is open")
        
check_socket("127.0.0.1", 6379)

### Find the real path of the `env_data` package (should be copied by tox)

In [ ]:
with ir.path("env_data.tests", "test_001.pkl") as oPath:
    sPath = oPath
print(type(sPath), sPath)

In [ ]:
sDirRoot = "/" + "/".join(sPath.parts[1:-1] + ("service_test",""))
sDirRoot

### Clear any old redis keys

In [ ]:
oRedis = redis.Redis()

In [ ]:
lKeys = oRedis.keys("flatland*")
lKeys

In [ ]:
for sKey in lKeys:
    print("Deleting:", sKey)
    oRedis.delete(sKey)

### Remove `/tmp/output.csv`

In [ ]:
!rm -f /tmp/output.csv

### kill any old `service.py` process

In [ ]:
!ps -ef | grep -i python | grep -i flatland.evaluators.service | awk '{print $2}' | xargs kill

In [ ]:
osEnv2 = os.environ.copy()

### Set some short timeouts for testing

In [ ]:
osEnv2["FLATLAND_OVERALL_TIMEOUT"]="10"
osEnv2["FLATLAND_PER_STEP_TIMEOUT"] = "2"
osEnv2["FLATLAND_MAX_SUCCESSIVE_TIMEOUTS"] = "2"
 
osEnv2["TEST_ID_FILTER"] = "Test_1"

### Create the python command for `service.py`

In [ ]:
FLATLAND_RL_SERVICE_ID = uuid.uuid4()
#sCmd = "python -m flatland.evaluators.service --test_folder ../env_data/tests/service_test --mergeDir ./tmp/merge --actionDir ./tmp/actions --pickle --missingOnly --service_id {FLATLAND_RL_SERVICE_ID}"
#sCmd = "python -m flatland.evaluators.service --test_folder ../env_data/tests/service_test --pickle --service_id {FLATLAND_RL_SERVICE_ID}" # --verbose"
sCmd = f"python -m flatland.evaluators.service --test_folder {sDirRoot} --pickle --service_id {FLATLAND_RL_SERVICE_ID}" # --verbose"
lsCmd = shlex.split(sCmd)
print(sCmd)
print(lsCmd)

### Run the command with Popen (output goes to jupyter stdout not notebook)

In [ ]:
with open("/tmp/stdout.txt","w") as out, open("/tmp/stderr.txt","w") as err:
    oPipe = sp.Popen(lsCmd, env=osEnv2, stdout=out,stderr=err)

In [ ]:
oPipe.poll()

In [ ]:
oFRC = FlatlandRemoteClient(test_env_folder=sDirRoot, verbose=False, use_pickle=True, flatland_rl_service_id=FLATLAND_RL_SERVICE_ID)

In [ ]:
env, env_dict = RailEnvPersister.load_new(f"{sDirRoot}/Test_0/Level_0.pkl")

In [ ]:
def forward_only_controller(obs, _env):
    dAct = {}
    for iAg in range(len(_env.agents)):
        dAct[iAg] = RailEnvActions.MOVE_FORWARD
    return dAct

def random_controller(obs, _env):
    dAct = {}
    for iAg in range(len(_env.agents)):
        dAct[iAg] = np.random.randint(0, 5)
    return dAct

In [ ]:
oObsB = DummyObservationBuilder()

In [ ]:
oObsB.get()

In [ ]:
def run_submission(slow_ep=1, delay=2, collect=False, verify=False):
    episode = 0
    obs = True
    while obs:
        print("==============")
        print(f"Episode : {episode} (1)")
        print("==============")
        obs, info = oFRC.env_create(obs_builder_object=oObsB)
        oFRC.env.record_steps = True
        if not obs:
            print("null observation - all envs completed!")
            break
        print("==============")
        print(f"Episode : {episode} (2)")
        print("==============")
        

        print(oFRC.env.dones['__all__'])

        if verify:
            with open(f"episode_{episode}.pkl", "rb") as file_in:
                expected = pickle.loads(file_in.read())
        for step in range(oFRC.env._max_episode_steps):
            if episode < 3:
                action = forward_only_controller(obs, oFRC.env)
            else:
                action = random_controller(obs, oFRC.env)
            
            time_start = time.time()
            
            if (episode == slow_ep) and (oFRC.env._elapsed_steps > 10):
                time.sleep(2)
                
            observation, all_rewards, done, info = oFRC.env_step(action)
            time_diff = time.time() - time_start
            if verify:
                assert expected[step] == oFRC.env.cur_episode[step]
            if collect:
                with open(f"episode_{episode}.pkl", "wb") as file_out:
                    data = pickle.dumps(oFRC.env.cur_episode)
                    file_out.write(data)
            if done['__all__']:
                if verify:
                    assert len(expected[step]) == len(oFRC.env.cur_episode[step])
                print("\nCompleted Episode : ", episode)
                print("Reward : ", sum(list(all_rewards.values())))
                break
            
        episode += 1
        
    print(f"Evaluation Complete - episodes={episode} - send submit message...")
    print(oFRC.submit())
    print("All done.")

In [ ]:
try:
    run_submission()
except Exception as timeoutException:
    print("Timed out.")
    print(timeoutException)
    try:
        # give evaluator enough time before submitting!
        time.sleep(2)
        print(f"Evaluation timed out - send submit message...")
        print(oFRC.submit())
        
    except Exception:
        print("All done.")

In [ ]:
!cat /tmp/stdout.txt

In [ ]:
!cat /tmp/stderr.txt

### Kill the evaluator process we started earlier

In [ ]:
!ps -ef | grep -i python | grep -i flatland.evaluators.service | awk '{print $2}' | xargs kill

In [ ]:
df = pd.read_csv("/tmp/output.csv").T
df

In [ ]:
df.transpose()["reward"].tolist()

In [ ]:
# Reward hard-coded. Due to speed 1./3. in speed ratio map (instead of 0.33), line generator outputs different earliest/latest than with 0.33.
assert np.array_equal(df.transpose()["reward"].tolist(), [np.nan, np.nan, -593.0, 1484.0], equal_nan=True)